In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, LayerNormalization, Dropout, Input, Embedding, BatchNormalization
from tensorflow.keras.models import Model, Sequential

In [2]:
df = pd.read_csv("./prototype_data.csv")

In [3]:
df.head()

,일시,PM-10,"('풍속', 98)","('풍속', 99)","('풍속', 108)","('풍속', 112)","('풍속', 119)","('풍속', 133)","('풍속', 201)","('강수량', 98)",...,month_11,month_12,hour_00,hour_03,hour_06,hour_09,hour_12,hour_15,hour_18,hour_21
0,2023-01-03 03:00:00,27.0,-1.001528,-1.276354,-0.794339,0.077853,-0.865018,-0.777845,-1.041770,-0.194231,...,False,False,False,True,False,False,False,False,False,False
1,2023-01-03 06:00:00,29.0,-1.001528,-1.178118,-0.692980,-0.654746,-1.128880,-0.777845,-1.209365,-0.194231,...,False,False,False,False,True,False,False,False,False,False
2,2023-01-03 09:00:00,30.0,-0.902350,-1.276354,-0.895697,-1.533866,-1.304788,-0.863775,-0.957973,-0.194231,...,False,False,False,False,False,True,False,False,False,False
3,2023-01-03 12:00:00,41.0,-0.604817,-0.981647,-0.084830,-0.508227,-0.513201,-0.262265,-0.957973,-0.194231,...,False,False,False,False,False,False,True,False,False,False
4,2023-01-03 15:00:00,31.0,0.585315,-0.294000,0.827396,0.297633,0.982019,1.198545,0.466585,-0.194231,...,False,False,False,False,False,False,False,True,False,False


In [4]:
df2 = df.set_index("일시")

In [5]:
x = df2.drop("PM-10", axis = 1)
y = df2["PM-10"]

In [6]:
# 시계열 예측과 같은 작업에서는 단일 스택을 사용하여 입력 시퀀스를 처리
def create_seq2seq_data(X, y, input_steps=52, output_steps=24): # input_steps는 우리가 예측하는 시간이고 output_steps는 예측할 시간(output_steps는 고정)
    Xs, ys = [], []
    for i in range(len(X) - input_steps - output_steps):
        Xs.append(X.iloc[i:(i + input_steps)].values)
        ys.append(y.iloc[(i + input_steps):(i + input_steps + output_steps)].values)
    return np.array(Xs).astype(np.float32), np.array(ys).astype(np.float32)

input_steps = 52
output_steps = 24
X_seq2seq, y_seq2seq = create_seq2seq_data(x, y, input_steps, output_steps)

x_train, x_test, y_train, y_test = train_test_split(X_seq2seq, y_seq2seq, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [7]:
# 트랜스포머 모델 정의 - 트랜스포머 모델의 입력 데이터 형식은 (batch_size, sequence_length, num_features)
class MultiHeadSelfAttention(Layer): # 입력으로 주어진 시퀀스를 각 헤드에 대해 독립적으로 어텐션을 수행하고, 그 결과를 결합하여 출력(병렬구조)
    def __init__(self, embed_dim, num_heads=8): # embed_dim은 입력 임베딩 차원, num_heads는 어텐션을 수행할 헤드의 수
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0: # 입력 임베딩 차원이 헤드의 수로 나누어 떨어지지 않으면 ValueError가 발생
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads # 각 헤드의 프로젝션 차원은 embed_dim // num_heads
        # query_dense, key_dense, value_dense 및 combine_heads는 각각 어텐션에 사용되는 dense 레이어
        self.query_dense = Dense(embed_dim)
        self.key_dense = Dense(embed_dim)
        self.value_dense = Dense(embed_dim)
        self.combine_heads = Dense(embed_dim)

    # 정보 손실 문제 해결
    def attention(self, query, key, value): # attention 함수는 주어진 쿼리, 키 및 값에 대해 어텐션 가중치를 계산하고, 이를 사용하여 값을 가중합하여 출력을 생성
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size): # separate_heads 함수는 입력을 여러 헤드로 분리하는 메서드
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim)) # 입력 형태는 (batch_size, num_heads, seq_len, projection_dim) 형태로 변환
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs): # 입력에 대한 어텐션을 계산하는 메서드
        batch_size = tf.shape(inputs)[0]
        # 입력에 대해 query_dense, key_dense, value_dense 레이어를 통과시키고, 그 결과를 각각 separate_heads() 메서드로 분리
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        attention, weights = self.attention(query, key, value) # 분리된 쿼리, 키 및 값에 대해 어텐션을 계산하고, 이를 결합
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim)) 
        output = self.combine_heads(concat_attention) # 마지막으로, 결합된 어텐션 결과에 combine_heads 레이어를 적용하여 최종 출력을 생성
        return output

In [8]:
class TransformerBlock(Layer): # TransformerBlock은 여러 층으로 쌓여 전체 트랜스포머 모델을 형성하며, 입력 시퀀스의 특징을 추출하고 다음 층으로 전달
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1): # embed_dim은 입력 임베딩 차원, num_heads는 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수
        # ff_dim은 피드 포워드 신경망의 은닉층 크기, rate는 dropout의 비율
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads) # att는 멀티 헤드 셀프 어텐션 레이어를 생성
        self.ffn = Sequential( # ffn은 피드 포워드 신경망을 생성. 이는 두 개의 dense 레이어로 구성되어 있음. 
            [Dense(ff_dim, activation="relu"), Dense(embed_dim),] # 첫 번째 레이어는 ReLU 활성화 함수를 사용하고, 두 번째 레이어는 선형 활성화 함수를 사용
        )
        # layernorm1, layernorm2는 레이어 정규화 레이어
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        # dropout1, 2 레이어도 설정
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False): # 입력을 받아와서 먼저 멀티 헤드 셀프 어텐션을 수행
        # 어텐션 출력에 드롭아웃을 적용하고, 입력과 어텐션 출력을 더한 후 레이어 정규화를 수행
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        # 그 다음, 피드 포워드 신경망을 통해 처리된 출력에 드롭아웃을 적용
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        # 마지막으로, 이전 출력과 더한 후 다시 레이어 정규화를 수행하여 최종 출력을 생성
        return self.layernorm2(out1 + ffn_output)

In [9]:
class TokenAndPositionEmbedding(Layer): # 트랜스포머 모델의 입력으로 사용될 토큰 및 위치 임베딩을 생성하는 레이어를 정의
                                        # 입력 시퀀스의 각 토큰에 대한 임베딩과 해당 위치에 대한 임베딩을 결합하여 최종 임베딩을 생성
    def __init__(self, maxlen, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.maxlen = maxlen # maxlen은 입력 시퀀스의 최대 길이
        self.embed_dim = embed_dim # embed_dim은 토큰 및 위치 임베딩의 차원
        self.token_emb = Dense(embed_dim) # token_emb는 입력 토큰에 대한 임베딩을 생성하는 밀집 레이어
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim) # pos_emb는 입력 시퀀스의 각 위치에 대한 임베딩을 생성하는 임베딩 레이어
                                                                         # 위치 임베딩의 크기는 (maxlen, embed_dim)

    def call(self, x): # 입력으로 주어진 토큰 시퀀스에 대해 위치 임베딩을 생성
        maxlen = tf.shape(x)[-2] 
        positions = tf.range(start=0, limit=maxlen, delta=1)
        # 각 위치에 대한 임베딩은 Embedding 레이어를 사용하여 생성되며, 시퀀스의 각 위치에 해당하는 임베딩을 가져와서 positions 변수에 저장
        positions = self.pos_emb(positions)
        token_embeddings = self.token_emb(x) # 입력 토큰에 대한 임베딩은 token_emb 레이어를 사용하여 생성
        return token_embeddings + positions # 위치 임베딩과 토큰 임베딩을 더하여 최종 임베딩을 생성하고 반환

# 문장을 한번에 병렬 처리해버리는 Transformer에 단어 순서를 알려주기 위한 작업
# 이 모델을 사용하면 입력 시퀀스의 각 토큰에 대해 임베딩을 생성하고, 해당 토큰의 위치 정보를 포함하여 트랜스포머 모델에 입력으로 전달

In [10]:
def build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks): # 트랜스포머 모델을 구축하는 데 사용
    # 모델의 입력 형태, 출력의 스텝 수, 임베딩 차원, 어텐션 헤드 수, 피드 포워드 신경망의 은닉층 크기, 트랜스포머 블록의 수
    inputs = Input(shape=input_shape) # 입력 데이터를 받기 위한 Keras의 Input 레이어를 생성
    embedding_layer = TokenAndPositionEmbedding(input_shape[0], embed_dim) # TokenAndPositionEmbedding 클래스를 사용하여 입력 데이터에 토큰 및 위치 임베딩을 적용
    x = embedding_layer(inputs)
    for _ in range(num_transformer_blocks): # 지정된 개수의 트랜스포머 블록을 생성하고 이전 블록의 출력을 현재 블록의 입력으로 전달
        x = TransformerBlock(embed_dim, num_heads, ff_dim)(x) # 모든 트랜스포머 블록이 생성된 후, 출력을 하나의 값으로 예측하기 위해 Dense 레이어를 적용
    x = Dense(1)(x)  # 수정된 부분: Dense(1)을 사용하여 각 타임 스텝에 대해 하나의 값을 예측
    x = tf.keras.layers.Flatten()(x)  # 수정된 부분: 출력을 평탄화하여 맞는 차원으로 변환
    outputs = Dense(output_steps)(x)  # 최종 출력 레이어
    model = Model(inputs=inputs, outputs=outputs) # 출력을 평탄화하여 차원을 맞추고, 최종 출력 레이어를 추가하여 최종 모델을 생성
    return model

In [11]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기
num_transformer_blocks = 2

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("./transformer-model.keras", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 52, 611)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding         │ (None, 52, 32)              │          21,248 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block (TransformerBlock) │ (None, 52, 32)              │           6,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_1                  │ (None, 52, 32)              │           6,464 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 52, 1)               │              33 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 52)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 24)                  │           1,272 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 35,481 (138.60 KB)

 Trainable params: 35,481 (138.60 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# 모델 학습
history = model.fit(x_train, y_train, epochs=200, validation_data=(x_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(x_test, y_test)
predictions = model.predict(x_test)

Epoch 1/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 1790.0867 - val_loss: 1025.4814
Epoch 2/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 887.3341 - val_loss: 611.3013
Epoch 3/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 610.8051 - val_loss: 511.7403
Epoch 4/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 511.4307 - val_loss: 462.1253
Epoch 5/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 456.2718 - val_loss: 426.9697
Epoch 6/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 426.3393 - val_loss: 414.1013
Epoch 7/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 436.3538 - val_loss: 399.2045
Epoch 8/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 416.5264 - val_loss: 396.9168
Epoch 9/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 365.3359 - val_loss: 390.4256
Epoch 10/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 389.1505 - val_loss: 389.9708
Epoch 11/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 383.9828 - val_loss: 380.3276
Epoch 12/200
42/

In [13]:
predictions[:10]

array([[ 72.17364  ,  69.49634  ,  87.33026  ,  70.4688   ,  68.77903  ,
         77.41383  ,  61.996014 ,  55.128338 ,  68.04587  ,  81.88378  ,
         93.29213  ,  77.44663  ,  85.68073  , 101.974594 ,  99.31532  ,
        108.17283  ,  83.25248  ,  61.168198 ,  50.361984 ,  32.117596 ,
         30.6617   ,  32.784607 ,  31.603687 ,  31.568756 ],
       [ 20.861454 ,  16.420088 ,  16.21215  ,   9.101044 ,  22.738459 ,
         22.133415 ,  38.392967 ,  60.65935  ,  90.98017  , 104.06591  ,
        105.86627  ,  76.68277  ,  68.85672  ,  65.20919  ,  60.796932 ,
         69.95755  ,  73.76267  ,  76.46447  ,  47.324593 ,  14.040215 ,
         22.741623 ,  24.973751 ,  47.435337 ,  99.24314  ],
       [ 31.39444  ,  38.05339  ,  32.158607 ,  22.321983 ,  19.875393 ,
         25.145008 ,  26.795355 ,  26.117208 ,  26.614246 ,  40.368652 ,
         27.895851 ,  23.545353 ,  18.08906  ,  24.201962 ,  30.798943 ,
         31.010073 ,  31.738043 ,  29.61192  ,  26.97863  ,  20.451904 ,
  

In [14]:
y_test[:10]

array([[ 70.,  69.,  86.,  70.,  69.,  77.,  60.,  54.,  67.,  80.,  91.,
         76.,  85.,  99.,  96., 104.,  80.,  57.,  49.,  32.,  28.,  30.,
         28.,  31.],
       [ 22.,  15.,  11.,   9.,  23.,  20.,  37.,  57.,  90., 105., 105.,
         76.,  71.,  67.,  65.,  73.,  75.,  80.,  49.,  15.,  22.,  28.,
         46., 102.],
       [ 26.,  38.,  33.,  23.,  19.,  23.,  28.,  28.,  24.,  39.,  27.,
         23.,  19.,  24.,  30.,  30.,  32.,  28.,  27.,  20.,  13.,  19.,
         30.,  23.],
       [ 56.,  45.,  44.,  28.,  32.,  38.,  23.,  25.,  44.,  60.,  39.,
         34.,  50.,  69.,  63.,  50.,  65.,  71.,  72.,  57.,  60.,  72.,
         65.,  63.],
       [ 15.,   9.,   5.,  15.,  14.,  17.,  15.,  19.,  16.,  15.,  13.,
          8.,   7.,   5.,   5.,   8.,  19.,  24.,  21.,  22.,  27.,  23.,
         24.,  25.],
       [ 60.,  57.,  46.,  42.,  57.,  62.,  59.,  37.,  43.,  42.,  46.,
         49.,  56.,  45.,  44.,  28.,  32.,  38.,  23.,  25.,  44.,  60.,
       

In [15]:
predictions.shape

(414, 24)